In [3]:
import pandas as pd
import os
import sys
import gpxpy.gpx
import gpxpy
import osmium as osm
import pathlib
path_to_harz = pathlib.Path(
    r"C:\Users\oleh.bakumenko\PythonWorkspace\PycharmProjects\sandbox\harz")
all_stamps = pd.read_excel(os.path.join(
    path_to_harz, 'excel', 'HWN2024_export_gpx.xlsx'))

In [4]:
all_stamps = pd.read_excel(os.path.join(
    path_to_harz, 'excel', 'HWN2024_export_gpx.xlsx'))

The goal is to get functionality to display the chosen waypoints (stamp points) from the excel list on the map. 
We downloaded maps of Niedersachsen and Sachsen-Anhalt together, because Harz occupies both regions.
The coordinates will be rounded to 4th digit. Using abstract variables in case we want to change cities later.
By looking on OpenStreetMap we picked border cities that we will take as corners for the map showing:


Left:
Seesen      (51.8905, 10.1708)
Up:
Osterwieck  (51.9680, 10.7136)
Right:
Mansfeld    (51.5936, 11.4538)
Below:
Kelbra      (51.4363, 11.0414)

In [5]:
# city data
left = ['Seesen', (51.8905, 10.1708)]
up = ['Osterwieck', (51.9680, 10.7136)]
right = ['Mansfeld', (51.5936, 11.4538)]
below = ['Kelbra', (51.4363, 11.0414)]

From this we can get the coordinates for the corners:

up-left:    (51.9680, 10.1708)

up-right:   (51.9680, 11.4538)

down-right: (51.4363, 11.4538)

down-left:  (51.4363, 10.1708)


In [6]:
# get coords
left_coords, up_coords, right_coords, below_coords = left[1], up[1], right[1], below[1]

# get corner points
up_left = (up_coords[0], left_coords[1])
up_right = (up_coords[0], right_coords[1])
down_right = (below_coords[0], right_coords[1])
down_left = (below_coords[0], left_coords[1])

# Print the corner points
print("up-left:   ", up_left)
print("up-right:  ", up_right)
print("down-right:", down_right)
print("down-left: ", down_left)

up-left:    (51.968, 10.1708)
up-right:   (51.968, 11.4538)
down-right: (51.4363, 11.4538)
down-left:  (51.4363, 10.1708)


In [19]:

# Define a handler to extract information from the .osm file
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.nodes = []
        self.ways = []
        self.relations = []

    def node(self, n):
        self.nodes.append({
            'id': n.id,
            'lat': n.location.lat,
            'lon': n.location.lon,
            'tags': n.tags
        })

    def way(self, w):
        self.ways.append({
            'id': w.id,
            'nodes': list(w.nodes),
            'tags': w.tags
        })

    def relation(self, r):
        self.relations.append({
            'id': r.id,
            'members': list(r.members),
            'tags': r.tags
        })


# Initialize the handler and apply it to the OSM file
path_to_niedersachsen = os.path.join(
    path_to_harz, 'osm', 'niedersachsen-latest.osm.pbf')
path_to_sachsen_anhalt = os.path.join(
    path_to_harz, 'osm', 'sachsen-anhalt-latest.osm.pbf')
path_to_berlin = os.path.join(
    path_to_harz, 'osm', 'berlin-latest.osm.pbf')
handler = OSMHandler()
handler.apply_file(path_to_berlin)

# Access parsed data
print(f"Nodes: {len(handler.nodes)}")
print(f"Ways: {len(handler.ways)}")
print(f"Relations: {len(handler.relations)}")

Nodes: 7245373
Ways: 1168409
Relations: 17467


In [16]:
import webbrowser
import folium

# Initialize a folium map centered around the data
# Extract latitudes and longitudes from the nodes list
latitudes = [node['lat'] for node in handler.nodes]  # Extract latitudes
longitudes = [node['lon'] for node in handler.nodes]  # Extract longitudes

# Calculate the center of the map
map_center = [sum(latitudes) / len(latitudes),
              sum(longitudes) / len(longitudes)]

# Create a folium map centered at the average node location
mymap = folium.Map(location=map_center, zoom_start=8)

# Add ways to the map
for way in handler.ways:
    # Extract node IDs for the current way
    way_nodes = way['nodes']  # This should contain IDs of nodes
    points = []
    for node_id in way_nodes:
        # Find the node's latitude and longitude using the node ID
        for node in handler.nodes:
            if node['id'] == node_id:
                # Append (lat, lon) to points
                points.append((node['lat'], node['lon']))
                break

    if len(points) > 1:  # Only plot ways with more than one node
        folium.PolyLine(points, color='blue', weight=2.5,
                        opacity=0.7).add_to(mymap)

# Add nodes as markers (optional)
for node in handler.nodes:
    folium.Marker([node['lat'], node['lon']], icon=folium.Icon(
        color='red', icon='info-sign')).add_to(mymap)

# Save the map as an HTML file
mymap.save("sachsen_anhalt_map.html")

# Open the map in the browser
webbrowser.open("sachsen_anhalt_map.html")

KeyboardInterrupt: 

In [20]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point

# Create lists to store geometries for GeoDataFrames
line_geometries = []
for way in handler.ways:
    way_nodes = way['nodes']  # This should be a list of node IDs
    points = []

    # Gather points for the way based on node IDs
    for node_id in way_nodes:
        # Find the corresponding node's lat/lon
        for node in handler.nodes:
            if node['id'] == node_id:
                points.append((node['lon'], node['lat']))  # (lon, lat) format
                break

    if len(points) > 1:  # Only create LineString if there are enough points
        line_geometries.append(LineString(points))

# Create GeoDataFrame for ways
gdf_ways = gpd.GeoDataFrame(geometry=line_geometries)

# Create GeoDataFrame for nodes
node_geometries = [Point(node['lon'], node['lat']) for node in handler.nodes]
gdf_nodes = gpd.GeoDataFrame(geometry=node_geometries)

# Plotting
fig, ax = plt.subplots(figsize=(12, 10))

# Plot lines (roads)
gdf_ways.plot(ax=ax, color='blue', linewidth=0.5, label='Roads')

# Plot points (nodes)
gdf_nodes.plot(ax=ax, color='red', markersize=1, label='Nodes')

# Set title and labels
ax.set_title("OSM Data Visualization")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()

# Show plot
plt.show()

KeyboardInterrupt: 

: 